In [ ]:
from typing import TypedDict
from langgraph.graph import START, StateGraph
from src.utils.models import get_model

RuntimeError: Called get_config outside of a runnable context

In [ ]:


model = get_model()


class State(TypedDict):
      topic: str
      joke: str
      poem: str


def write_joke(state: State):
    writer = get_stream_writer()
    writer(
        {
                "message_type": "tool_info",
                "message": f"Writing a joke about topic: {state['topic']}",
        }
    )
    topic = state["topic"]
    joke_response = model.invoke(
        [{"role": "user", "content": f"Write a joke about {topic}"}]
    )
    return {"joke": joke_response.content}


def write_poem(state: State):
    writer = get_stream_writer()
    writer(
            {
                "message_type": "tool_info",
                "message": f"Writing a poem about topic: {state['topic']}",
            }
    )
    topic = state["topic"]
    poem_response = model.invoke(
        [{"role": "user", "content": f"Write a short poem about {topic}"}]
    )
    return {"poem": poem_response.content}




graph = (
      StateGraph(State)
      .add_node(write_joke)
      .add_node(write_poem)
      # write both the joke and the poem concurrently
      .add_edge(START, "write_joke")
      .add_edge(START, "write_poem")
      .compile()
)



In [34]:
# The "messages" stream mode returns a tuple of (message_chunk, metadata)
# where message_chunk is the token streamed by the LLM and metadata is a dictionary
# with information about the graph node where the LLM was called and other information
for mode, chunk in graph.stream(
    {"topic": "cats"},
    stream_mode=["messages","computers"],  
):
    # Filter the streamed tokens by the langgraph_node field in the metadata
    # to only include the tokens from the write_poem node
    # if msg.content and metadata["langgraph_node"] == "write_poem":
    #     # print(msg.content, end="|", flush=True)
    #     print(msg)
    if mode == "custom":
        print(chunk.get("message"))
    if mode == "messages":
        msg, metadata = chunk
        if msg.content and metadata["langgraph_node"] == "write_poem":
            print(msg.content, end="|", flush=True)

In| twilight|’s| hush| a| whisk|ered| silhouette|  
|sl|ips| through| the| doorway|,| a| silent| silhouette|,|  
|soft| paws| whisper|ing| on| the| floor|—|  
|a| tiny| universe| in| a| p|urring| roar|.|  

|Eyes| like| lantern|s|,| amber| fire|flies| in| the| night|,|  
|they| curl| in| circles|,| claim| the| world| with| gentle| might|.|  

|A| moment| of| quiet|,| a| breath| in| the| air|—|  
|a| cat|,| a| mystery|,| an| endless| prayer|.|

In [21]:
think_start =False
for chunk in model.stream(content_data):
    think_content = chunk.additional_kwargs.get("reasoning_content",None)
    if think_content:
        if not think_start:
            think_start = True
            print("\n[THINKING...]\n", end="", flush=True)
        print(chunk.additional_kwargs["reasoning_content"], end="", flush=True)
    else:
        if think_start:
            think_start = False
            print("\n[END THINKING]\n", end="", flush=True)
        print(chunk.content, end="", flush=True)

NameError: name 'content_data' is not defined

In [10]:
msg

AIMessageChunk(content='', additional_kwargs={}, response_metadata={}, id='lc_run--df8f3b52-d230-4cab-9d84-8d44db562109', chunk_position='last')

In [8]:
metadata

{'langgraph_step': 1,
 'langgraph_node': 'write_poem',
 'langgraph_triggers': ('branch:to:write_poem',),
 'langgraph_path': ('__pregel_pull', 'write_poem'),
 'langgraph_checkpoint_ns': 'write_poem:98a8d4db-97d3-81f3-1491-9e8136a4389d',
 'checkpoint_ns': 'write_poem:98a8d4db-97d3-81f3-1491-9e8136a4389d',
 'ls_provider': 'ollama',
 'ls_model_name': 'gpt-oss',
 'ls_model_type': 'chat',
 'ls_temperature': None,
 'LANGSMITH_PROJECT': 'deep-research-agent',
 'LANGSMITH_TRACING': 'true',
 'LANGSMITH_ENDPOINT': 'https://api.smith.langchain.com',
 'revision_id': '7192b14-dirty'}

In [22]:
model.invoke("How are you?")

AIMessage(content='I’m just a bundle of code, so I don’t have feelings, but thanks for asking! How can I help you today?', additional_kwargs={}, response_metadata={'model': 'gpt-oss', 'created_at': '2025-12-13T16:50:35.019074Z', 'done': True, 'done_reason': 'stop', 'total_duration': 2814250750, 'load_duration': 1620707958, 'prompt_eval_count': 71, 'prompt_eval_duration': 208561334, 'eval_count': 79, 'eval_duration': 920647293, 'model_name': 'gpt-oss', 'model_provider': 'ollama'}, id='lc_run--8b770ebb-8206-441e-8e76-60c78fe08cc1-0', usage_metadata={'input_tokens': 71, 'output_tokens': 79, 'total_tokens': 150})

In [23]:
msg

AIMessageChunk(content='', additional_kwargs={}, response_metadata={}, id='lc_run--963419e5-e6a1-49a7-ad17-23d0f7dc5447', chunk_position='last')

# ollama search

In [35]:
import ollama

In [37]:
response = ollama.web_search("What is the capital of France?")

In [42]:
print(response.results[0].content)

Paris | Definition, Map, Population, Facts, & History | Britannica
**
**[![Encyclopedia Britannica](https://cdn.britannica.com/mendel/eb-logo/MendelNewThistleLogo.png)](https://www.britannica.com/)
Search Britannica**Click here to search**
[![Encyclopedia Britannica](https://cdn.britannica.com/mendel/eb-logo/MendelNewThistleLogo.png)](https://www.britannica.com/)
Search Britannica**Click here to search**
[SUBSCRIBE](https://premium.britannica.com/premium-membership/?utm_source=premium&utm_medium=global-nav&utm_campaign=blue-evergreen)
[SUBSCRIBE](https://premium.britannica.com/premium-membership/?utm_source=premium&utm_medium=global-nav-mobile&utm_campaign=blue-evergreen)
Login**
https://premium.britannica.com/premium-membership/?utm\_source=premium&utm_me&utm_medium=nav-login-box&utm_ca&utm_campaign=evergreen
****
[SUBSCRIBE](https://premium.britannica.com/premium-membership/?utm_source=premium&utm_medium=hamburger-menu&utm_campaign=blue)
* [Home](https://www.britannica.com/)
* [Histo

In [44]:
response.results[0].url

'https://www.britannica.com/place/Paris'

In [45]:
fetch = ollama.web_fetch(response.results[0].url)

In [52]:
print(fetch.content)

Paris | Definition, Map, Population, Facts, & History | Britannica
**
**[![Encyclopedia Britannica](https://cdn.britannica.com/mendel/eb-logo/MendelNewThistleLogo.png)](https://www.britannica.com/)
Search Britannica**Click here to search**
[![Encyclopedia Britannica](https://cdn.britannica.com/mendel/eb-logo/MendelNewThistleLogo.png)](https://www.britannica.com/)
Search Britannica**Click here to search**
[SUBSCRIBE](https://premium.britannica.com/premium-membership/?utm_source=premium&utm_medium=global-nav&utm_campaign=blue-evergreen)
[SUBSCRIBE](https://premium.britannica.com/premium-membership/?utm_source=premium&utm_medium=global-nav-mobile&utm_campaign=blue-evergreen)
Login**
https://premium.britannica.com/premium-membership/?utm\_source=premium&utm_me&utm_medium=nav-login-box&utm_ca&utm_campaign=evergreen
****
[SUBSCRIBE](https://premium.britannica.com/premium-membership/?utm_source=premium&utm_medium=hamburger-menu&utm_campaign=blue)
* [Home](https://www.britannica.com/)
* [Histo

In [51]:
model.

TypeError: 'ChatOllama' object is not subscriptable

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size=4000, chunk_overlap=200)
docs = splitter.split_text(long_text)

In [53]:
len(fetch.content.split(" "))

2117

In [54]:
"How.  are you.     ?".split(" ")

['How.', '', 'are', 'you.', '', '', '', '', '?']

In [56]:
docs = split_text_by_words(fetch.content, chunk_size=1000, overlap_size=50)

In [58]:
len(docs[0].split(" "))

1000

In [59]:
len(docs)

3

In [65]:
from src.deep_research_agent.tools.search_tool import summarize_long_content

In [68]:
res = await summarize_long_content(fetch.content)

In [70]:
print(res.summary)

Paris, the capital of France, sits on the Seine in the north‑central part of the country. The city’s 20 arrondissements form the core of the Île‑de‑France metropolitan area, home to ~13 million people and covering ~15,000 km². Paris is a UNESCO World Heritage Site famous for landmarks such as the Eiffel Tower, Notre‑Dame, the Louvre, Arc de Triomphe, and the Latin Quarter. The city has been a political, cultural, and economic hub since the 12th century, witnessing pivotal events like the French Revolution (1789) and the 19th‑century Haussmann renovation that created its iconic boulevards and public parks. Today, Paris balances preservation of its heritage with contemporary urban development, remaining a leading center for art, fashion, technology, and diplomacy.

Green infrastructure focuses on the Seine and surrounding parks. The Seine, 13 km long, cuts through 10 of the 20 arrondissements, flowing from southeast to northwest and southwest. At water level (~30 ft below street level) t

In [71]:
print(res.key_excerpts)

Paris is the capital and most populous city of France, located on the Seine in the north-central part of the country.
It is the political, cultural, and economic heart of France and a major global metropolis, ranking among the world’s leading tourist destinations.
The city’s core is often called the City of Light and it has earned a reputation as a center of art, fashion, gastronomy, and history.
Paris is divided into 20 municipal arrondissements, each with its own character.
The city is a UNESCO World Heritage Site, home to iconic landmarks such as the Eiffel Tower, Notre-Dame Cathedral, the Louvre Museum, Arc de Triomphe, and the Latin Quarter.
At water level, some 30 feet (9 metres) below street level, the river is bordered—at least on those portions not transformed into expressways—by cobbled quays graced with trees and shrubs.; It was the world’s first elevated park (first phase completed in 1994) and the first ‘green space’ constructed on a viaduct; it has since inspired other ci

In [ ]:
from langchain.tools import tool

@tool
def add_two_numbers(a: int, b: int) -> str:
    """Add two numbers.

    Args:
        a: The first number.
        b: The second number.

    Returns:
        The sum of the two numbers as a string.
    """
    return str(a + b)

model = model.bind(tools=[add_two_numbers])


In [90]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="What is 5 + 7?")])

ValidationError: 1 validation error for Tool
  Input should be a valid dictionary or instance of Tool [type=model_type, input_value=StructuredTool(name='add_...numbers at 0x11d597530>), input_type=StructuredTool]
    For further information visit https://errors.pydantic.dev/2.12/v/model_type

In [91]:
from langchain.tools import tool, ToolRuntime

In [96]:
ToolRuntime(state={"visited_urls": []},context=None,config=None,stream_writer=None,tool_call_id="test_tool_call_id",store=None)

ToolRuntime(state={'visited_urls': []}, context=None, config=None, stream_writer=None, tool_call_id='test_tool_call_id', store=None)

In [97]:
def generate_numbers(n: int):
    for i in range(n):
        yield i

    return "done"

In [98]:
for val in generate_numbers(10):
    print(val)

0
1
2
3
4
5
6
7
8
9


In [99]:
val

9

In [120]:
from src.utils.helpers import get_prompt_template, get_today_str
from src.config import ROOT_DIR
from src.utils.models import get_model
from langchain_core.runnables import RunnableConfig


template = get_prompt_template(
            ROOT_DIR / "src/deep_research_agent/prompts/research_agent_system_prompt.jinja"
        )
prompt = template.render(interleaved_thinking=True,date=get_today_str(),
           )
print(prompt)

You are a research assistant conducting research on the user's input topic. For context, today's date is Thu Dec 25, 2025.

<Task>
Your job is to use tools to gather information about the user's input topic.
You can use any of the tools provided to you to find resources that can help answer the research question. You can call these tools in series or in parallel, your research is conducted in a tool-calling loop.
</Task>

<Available Tools>
You have access to two main tools:
1. **web_search**: For conducting web searches to gather information
2. **think_tool**: For reflection and strategic planning during research.After each `web_search` tool call, use `think_tool` to analyze the results,

**CRITICAL: Use think_tool after each search to reflect on results and plan next steps**
</Available Tools>

<Instructions>
Think like a human researcher with **limited time**. Follow these steps:

1. **Read the question carefully** - What specific information does the user need?
2. **Start with broad

In [121]:
l = [1,2,3,4,5]
l[:-1]

[1, 2, 3, 4]